# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [9]:
# Define the generator function
def square_root_generator():
    n = 1
    while True:
        yield n ** 0.5
        n += 1

# Example usage:
generator = square_root_generator()

# Initialize variables for sum and count
sum_of_outputs = 0
count = 0
thirteenth_number = None

# Iterate through the generator
for sqrt_value in generator:
    sum_of_outputs += sqrt_value
    count += 1
    if count == 13:
        thirteenth_number = sqrt_value
        break  # Exit the loop after finding the 13th number

# Answering the questions
# Question 1: Calculate the sum of the outputs for limit = 5
sum_limit_5 = sum_of_outputs

# Output the results
print("Sum of the outputs for limit = 5:", sum_limit_5)
print("13th number yielded:", thirteenth_number)

Sum of the outputs for limit = 5: 32.854555867161245
13th number yielded: 3.605551275463989


8.382332347441762

3.605551275463989

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [13]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [15]:
import duckdb

# Define the first generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# Define the second generator
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Create a connection to DuckDB
conn = duckdb.connect(database=':memory:', read_only=False)

# Load the first generator and create a table
conn.execute("CREATE TABLE people (ID INTEGER, Name VARCHAR, Age INTEGER, City VARCHAR, Occupation VARCHAR)")

# Populate the table with data from the first generator
for person in people_1():
    conn.execute("INSERT INTO people VALUES (?, ?, ?, ?, NULL)", list(person.values()))

# Calculate the sum of ages of all people before appending the second generator
result_before_append = conn.execute("SELECT SUM(Age) FROM people").fetchall()

# Append the data from the second generator to the same table
for person in people_2():
    conn.execute("INSERT INTO people VALUES (?, ?, ?, ?, ?)", list(person.values()))

# Calculate the sum of ages of all people after appending the second generator
result_after_append = conn.execute("SELECT SUM(Age) FROM people").fetchall()

# Output the results
print("Sum of ages of all people before appending the second generator:", result_before_append[0][0])
print("Sum of ages of all people after appending the second generator:", result_after_append[0][0])


Sum of ages of all people before appending the second generator: 140
Sum of ages of all people after appending the second generator: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [18]:
%%capture
!pip install dlt[duckdb]

In [16]:
import duckdb

# Define the first generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

# Define the second generator
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Create a connection to DuckDB
conn = duckdb.connect(database=':memory:', read_only=False)

# Load the first generator and create a table with a primary key on ID
conn.execute("CREATE TABLE people (ID INTEGER PRIMARY KEY, Name VARCHAR, Age INTEGER, City VARCHAR, Occupation VARCHAR)")

# Populate the table with data from the first generator
for person in people_1():
    conn.execute("INSERT INTO people VALUES (?, ?, ?, ?, NULL)", list(person.values()))

# Merge the data from the second generator with the existing table, replacing records with overlapping IDs
for person in people_2():
    conn.execute("INSERT OR REPLACE INTO people VALUES (?, ?, ?, ?, ?)", list(person.values()))

# Verify the number of records and the age of ID 3
result = conn.execute("SELECT COUNT(*), SUM(Age) FROM people WHERE ID = 3").fetchone()

# Output the results
print("Total number of records:", result[0])
print("Sum of ages of all people:", result[1])

Total number of records: 1
Sum of ages of all people: 33


In [19]:
# Calculate the sum of ages of all people loaded
total_age_sum = conn.execute("SELECT SUM(Age) FROM people").fetchone()[0]

# Output the result
print("Sum of ages of all the people loaded:", total_age_sum)

Sum of ages of all the people loaded: 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX